In [1]:
import pandas as pd

## Data Integration

In [2]:
df_basic = pd.read_csv('OfflineFiles/IMDB/title_basics.tsv',sep='\t')
df_rating = pd.read_csv('OfflineFiles/IMDB/title_ratings.tsv',sep='\t')
df_akas = pd.read_csv('OfflineFiles/IMDB/title.akas.tsv',sep='\t')

/var/folders/nl/jzl2sdyd5g7gv2bk8lfg29tr0000gn/T/ipykernel_16942/3045235494.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_basic = pd.read_csv('OfflineFiles/IMDB/title_basics.tsv',sep='\t')
/var/folders/nl/jzl2sdyd5g7gv2bk8lfg29tr0000gn/T/ipykernel_16942/3045235494.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_akas = pd.read_csv('OfflineFiles/IMDB/title.akas.tsv',sep='\t')


In [3]:
df_basic.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
set(df_basic['titleType'].tolist())

{'movie',
 'short',
 'tvEpisode',
 'tvMiniSeries',
 'tvMovie',
 'tvPilot',
 'tvSeries',
 'tvShort',
 'tvSpecial',
 'video',
 'videoGame'}

In [5]:
df_movie = df_basic.loc[df_basic['titleType'] == 'movie']
df_movie.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama


In [6]:
df_movie_pre = df_movie[['tconst', 'primaryTitle', 'startYear', 'genres']]
df_movie_pre.set_index('tconst', inplace=True)

In [7]:
df_rating.set_index('tconst', inplace=True)

In [8]:
df_allSet = pd.concat([df_movie_pre, df_rating.reindex(df_movie_pre.index)], axis=1)

In [9]:
df_allSet.head()

,primaryTitle,startYear,genres,averageRating,numVotes
tconst,,,,,
tt0000009,Miss Jerry,1894,Romance,5.3,198.0
tt0000502,Bohemios,1905,\N,4.5,14.0
tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",6.0,778.0
tt0000591,The Prodigal Son,1907,Drama,4.7,19.0
tt0000615,Robbery Under Arms,1907,Drama,4.5,23.0


## Drop Nan

In [10]:
df_allSet.dropna(inplace=True)
df_dropN = df_allSet[df_allSet['genres'] != "\\N"]
df_dropN = df_dropN[df_dropN['startYear'] != "\\N"]


In [11]:
title = df_dropN['primaryTitle']
title

tconst
tt0000009                     Miss Jerry
tt0000574    The Story of the Kelly Gang
tt0000591               The Prodigal Son
tt0000615             Robbery Under Arms
tt0000630                         Hamlet
                        ...             
tt9916190                      Safeguard
tt9916270       Il talento del calabrone
tt9916362                          Coven
tt9916428            The Secret of China
tt9916538            Kuambil Lagi Hatiku
Name: primaryTitle, Length: 269697, dtype: object

In [12]:
title.to_csv('Data/title.csv')

In [13]:
# df_dropN.drop(columns='primaryTitle', inplace=True)
df_dropN.to_csv('Data/movie.csv')

## Categorical Data Processing

In [14]:
df = pd.read_csv('Data/movie.csv')
df.set_index('tconst', inplace = True)
df


,primaryTitle,startYear,genres,averageRating,numVotes
tconst,,,,,
tt0000009,Miss Jerry,1894,Romance,5.3,198.0
tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",6.0,778.0
tt0000591,The Prodigal Son,1907,Drama,4.7,19.0
tt0000615,Robbery Under Arms,1907,Drama,4.5,23.0
tt0000630,Hamlet,1908,Drama,3.9,25.0
...,...,...,...,...,...
tt9916190,Safeguard,2020,"Action,Adventure,Thriller",3.6,235.0
tt9916270,Il talento del calabrone,2020,Thriller,5.8,1347.0
tt9916362,Coven,2020,"Drama,History",6.4,4788.0


In [15]:
all_genres=set()
for line in df['genres']:
    line_s=line.split(",")
    for gen in line_s:
        all_genres.add(gen)
print(all_genres)

{'War', 'Romance', 'Animation', 'Music', 'Drama', 'Western', 'Horror', 'Game-Show', 'Adult', 'Adventure', 'Fantasy', 'Sci-Fi', 'Musical', 'Film-Noir', 'Thriller', 'Crime', 'Sport', 'Reality-TV', 'Short', 'Talk-Show', 'Biography', 'Family', 'Mystery', 'Comedy', 'Action', 'History', 'News', 'Documentary'}


In [16]:
all_genres_df=pd.DataFrame(0,index=df.index,columns=list(all_genres))
for i in range(len(df.index)):
    gens=df['genres'][i].split(",")
    for gen in gens:
        all_genres_df[gen][i]=1
all_genres_df
    

,War,Romance,Animation,Music,Drama,Western,Horror,Game-Show,Adult,Adventure,...,Short,Talk-Show,Biography,Family,Mystery,Comedy,Action,History,News,Documentary
tconst,,,,,,,,,,,,,,,,,,,,,
tt0000009,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000574,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
tt0000591,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000615,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000630,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9916190,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
tt9916270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt9916362,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
movies = pd.concat([df, all_genres_df], axis=1)
movies.drop(columns='genres', inplace=True)
movies

,primaryTitle,startYear,averageRating,numVotes,War,Romance,Animation,Music,Drama,Western,...,Short,Talk-Show,Biography,Family,Mystery,Comedy,Action,History,News,Documentary
tconst,,,,,,,,,,,,,,,,,,,,,
tt0000009,Miss Jerry,1894,5.3,198.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000574,The Story of the Kelly Gang,1906,6.0,778.0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
tt0000591,The Prodigal Son,1907,4.7,19.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
tt0000615,Robbery Under Arms,1907,4.5,23.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
tt0000630,Hamlet,1908,3.9,25.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9916190,Safeguard,2020,3.6,235.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
tt9916270,Il talento del calabrone,2020,5.8,1347.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt9916362,Coven,2020,6.4,4788.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
movies.to_csv('Data/movies_allGenres.csv')


## Samples

In [19]:
movies = pd.read_csv('Data/movies_allGenres.csv')
movies.set_index('tconst', inplace = True)


In [20]:
#def movies_sampling(movies):
movies_sample = movies[movies['numVotes'] > 0.01 * movies['numVotes'].max()]
movies_sample.shape

(5697, 32)

In [21]:
movies_sample

,primaryTitle,startYear,averageRating,numVotes,War,Romance,Animation,Music,Drama,Western,...,Short,Talk-Show,Biography,Family,Mystery,Comedy,Action,History,News,Documentary
tconst,,,,,,,,,,,,,,,,,,,,,
tt0010323,The Cabinet of Dr. Caligari,1920,8.0,63225.0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
tt0012349,The Kid,1921,8.3,125412.0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
tt0013442,Nosferatu,1922,7.9,96271.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0015324,Sherlock Jr.,1924,8.2,49294.0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
tt0015648,Battleship Potemkin,1925,8.0,57440.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9844522,Escape Room: Tournament of Champions,2021,5.7,42787.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
tt9848626,Hotel Transylvania 4: Transformania,2022,6.0,28554.0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
tt9866072,Holidate,2020,6.1,63866.0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
movies_sample[['startYear', 'averageRating', 'numVotes']].max()

startYear           2022.0
averageRating          9.3
numVotes         2616941.0
dtype: float64

In [23]:
movies_sample[['startYear', 'averageRating', 'numVotes']].min()

startYear         1920.0
averageRating        1.0
numVotes         26181.0
dtype: float64

## Normalize 

In [24]:
def movies_normalize(movies):
    movies_norm_col =  movies[['startYear', 'averageRating', 'numVotes']]
    movies[['startYear', 'averageRating', 'numVotes']] = (movies_norm_col - movies_norm_col.mean()) / (movies_norm_col.max() - movies_norm_col.min())# movies_norm_col.std()

    return movies

movies = movies_sample.copy()

normalized_movies = movies_normalize(movies)
normalized_movies

,primaryTitle,startYear,averageRating,numVotes,War,Romance,Animation,Music,Drama,Western,...,Short,Talk-Show,Biography,Family,Mystery,Comedy,Action,History,News,Documentary
tconst,,,,,,,,,,,,,,,,,,,,,
tt0010323,The Cabinet of Dr. Caligari,-0.809778,0.153399,-0.029707,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
tt0012349,The Kid,-0.799974,0.189544,-0.005704,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
tt0013442,Nosferatu,-0.790170,0.141351,-0.016952,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0015324,Sherlock Jr.,-0.770562,0.177496,-0.035084,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
tt0015648,Battleship Potemkin,-0.760758,0.153399,-0.031940,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9844522,Escape Room: Tournament of Champions,0.180418,-0.123709,-0.037596,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
tt9848626,Hotel Transylvania 4: Transformania,0.190222,-0.087565,-0.043090,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
tt9866072,Holidate,0.170614,-0.075516,-0.029460,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [25]:
normalized_movies.to_csv("Data/norm_sample_movies.csv")